In [125]:
import re

import numpy as np
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import re
import string

df = pd.read_csv('imdb_sup.csv')

In [259]:
#helper functions for cleaning
def remove_punc(word):
    punc = '''-!()[]{};:"\,<>./?@#$%^&*_~'''
    no_punc_word = word.translate(str.maketrans('', '', punc))
    return no_punc_word

def remove_BR(lst):
    words = []
    for word in lst:
        if word.endswith('br'):
            word = word[:-2]
        words.append(word)
    
    return ' '.join(words)

def make_lower(word):
    return word.lower()

def remove_numbers(lst):
    no_num_list = [x for x in lst if not any(c.isdigit() for c in x)]
    return ' '.join(no_num_list)

def remove_words(lst):
    extra_stops = ['aa', 'ab','br', 'us', 'mr', 'saw', 'until', 'no', 'when', 'with', 
            'like', 'just', 'even', 'it\'s', 'i\'m', 
            'who', 'i\'ve', 'what', 'he', 'see', 'up','get', 'been',
            'because', 'into', 'time', 'watch', 'â–','called', '2',
             '10', 'said','their', 'can','two', 'go', 'also', 'seen', 'him',
            'through', 'it', 'doesn\'t', 'you\'re', 'that\'s', 'there\'s',
            'come', 'said', 'all.', 'screen', 'person', 'i\'ll', 'is,'
            '5', 'sandra', 'them.', '3', '.', 'he\'s', 'man', 'they\'re',
            '\\\x96', '--', 'i\'d', 'is,', 'oh', 'one', 'much', 'movies',
            'say', '4', '1', 'five', 'what\'s', '15', 'ed', '...',
            'movie', 'film','', '-', 'people', 'could', 'make', 'films', 'reviews']


    stop_words = nltk.corpus.stopwords.words("english")
    stop_words = set(stop_words)
    stop_words.update(extra_stops) 

    words = [word for word in lst if word not in stop_words]

    return ' '.join(words)
    

In [163]:
def text_clean(message):
  message = remove_punc(message)   # remove punctiation except - and \'
  message = remove_words(message.split())
  message = remove_BR(message.split())
  message = remove_numbers(message.split())
  message = nltk.tag.pos_tag(message.split(), tagset='universal')
  message = [word for word,tag in message if (tag == 'ADJ' or tag == 'ADV' or tag == 'VERB')]
  message = ' '.join(message)
  message.lower()

  return message

In [164]:
test_str = "Skippy A wonderful wonderous wonder from \"Family Ties\" plays Eddie, a wussy 'metal' nerd who gets picked on. When his favorite wussy 'metal' singer, Sammi Curr, dies, he throws a hissy fit tearing down all the posters on his bedroom wall. But when he later gets an unreleased record that holds the spirit of his dead 'metal' idol. He first gets sucked into ideas of revenge, but then he doesn't want to take it as far as Sammi does. Which isn't really that far as his main victims only seem to go to the hospital. This movie is utterly laughable and has about as much to do with real metal as say, \"Rock Star\". OK, maybe a tad more than that piece of junk, but you get my point. And how ANYone can root for a guy played by Skippy from \"Family Ties\" I haven't a clue. The cameo by Gene Simmons is OK, and Ozzy Osbourne reaches coherency, I applaud him for that, but otherwise skip this one.<br /><br />My Grade: D <br /><br />Eye Candy:Elise Richards gets topless, an a topless extra at a pool party"
text_clean(test_str)

'Skippy wonderful wonderous wonder plays wussy nerd gets picked When favorite dies throws hissy fit tearing bedroom later gets unreleased holds spirit dead first gets sucked revenge want take far really far main seem utterly laughable real maybe tad piece played clue reaches applaud otherwise skip gets topless extra'

In [106]:
#cleans all reviews in dataset
df['Review'] = df['Review'].apply(text_clean)

In [165]:
#shuffles data entries
dff = df.sample(frac=1).reset_index(drop=True)

In [166]:
dff

,Review,Rating,Sentiment
0,this is the worst film I've seen in a long lon...,1,0
1,I tried to love this movie. I really did. Kevi...,4,0
2,This is going to be my first review on IMDb an...,1,0
3,"bad acting , combats are very awful , 3-4 seco...",2,0
4,This adaptation positively butchers a classic ...,1,0
...,...,...,...
49995,Fever Pitch is a sweet and charming addition t...,8,1
49996,"When it first came out, this work by the Meyse...",7,1
49997,Not very interesting teen whodunit saved from ...,4,0
49998,A surprising misfire from the usually reliable...,3,0


In [167]:
#stemming program
porterstemmer = PorterStemmer()

def steming(message):
  return[porterstemmer.stem(word) for word in message.split()]

In [266]:
train_X = dff['Review'].head(25000)  
train_Y = dff['Rating'].head(25000)  
test_X = dff['Review'].tail(25000) 
test_Y = dff['Rating'].tail(25000)

In [267]:
#tf idf
tf_idf = TfidfVectorizer(tokenizer=steming)
#applying tf idf to training data
X_train_tf = tf_idf.fit_transform(train_X)
#applying tf idf to training data
X_train_tf = tf_idf.transform(train_X)

In [254]:
print("n_samples: %d, n_features: %d" % X_train_tf.shape)

n_samples: 25000, n_features: 228862


In [268]:
#transforming test data into tf-idf matrix
X_test_tf = tf_idf.transform(test_X)

print("n_samples: %d, n_features: %d" % X_test_tf.shape) # the result of the test data and training data are the same

n_samples: 25000, n_features: 228862


In [269]:
#naive bayes classifier
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, train_Y)
#predicted y
y_pred = naive_bayes_classifier.predict(X_test_tf)

print(metrics.classification_report(test_Y, y_pred, target_names=[1,2,3,4,5,6,7,8]))
print("Confusion matrix:")
print(metrics.confusion_matrix(test_Y, y_pred))

ValueError: Number of classes, 8, does not match size of target_names, 2. Try specifying the labels parameter

In [264]:
test = 'First of all it was my childhood dream to see these hero\'s in a big screen and it was full filled. The movie was quite good for me but after getting out of the theatre there were not much scenes in my mind that i can feel awesome about. Zack Snyder is a master in making comic book style of movies he has proved it by his movie the Watchmen but in the movie the Justice league the movie lost its synder touch and the whole movie looks like a rip off to marvel movie mantra that they use in there films(like the happy go jolly characters not much tension on the screen light hearted) likewise Josh wedon has made this movie into a marvel like film. The Justice league had many character moments for all but the grip was loosen up to make the fan/audience glue to there seats. The whole colour tone of the movie was changed by josh and as the Wb asked them to cut down its length to 2hrs(as the internet says Wb asked them to do so) But anyone who watched the film can easily see that many of the scenes showned in the trailer is not in the movie,even the dialogue by steppen wolf "No protector here.. .. .   .... This world will fall like all the others" that one was also cut off in the film.in justice league the expensive reshooted scenes that have no connection to the actual film plot and at first report the movies run time was nearly 3hrs and now we got a 2hrs 1min movie to watch with most of the scenes cut off..As a movie lover its just hurts know that they reduce the length of the film that has so much potential to reach for more.When Bvs released the critics and some marvel fan boys said its so dark and lengthy with not much strong plot to establish the cause of the fight between Batman and superman. But after they release the extended cut bvs we all get to know the exact chronology of the plot. Likewise this movie should also release the extended cut/directors cut version but zack was great for making something that is so gripping and his tones delivers the theme itself but here its all josh wedon who just ruined Zack\'s vision and made a marvel-istic movie with his Cuts. Overall the movie was Great and All the actors played there roles perfectly and the image of batman had be shifted abit in my view as there is a bit of difference between the Bvs batman and the JL batman, Bvs batman was more like the exact comic book Dark Knight but in this He is more Like Tony stark here and there. And those who relay on F**king tomatoes ratings don\'t watch If you are a movie lover and if this is your genre then go for it From the intro scene of Batman you will love the movie While watching the films you won\'t be able to see any flaws because of its paced 2hrs cut.For me the rating for this film would be 6.5/10. Go and watch for yourself its an entertaining Flick don\'t miss it ..the only draw back you will have will be with its running time and scenes cut off.'

processed = []
processed.append(text_clean(test))

In [265]:
test_input = tf_idf.transform(processed)
test_input.shape

#0= bad review
#1= good review
res=naive_bayes_classifier.predict(test_input)[0]
if res==1:
    print("Good Review")
elif res==0:
    print("Bad Review")

Good Review
